# ICHI multiplayer - server-side

#### You can change the port the server is running on right here :

In [1]:
# This is the port of the server you want to run on
PORT = 8080

Server logic has been adaptated from this youtube video : https://www.youtube.com/watch?v=3QiPPX-KeSc

In [2]:
# import of all dependencies needed for the project
import socket
import threading
import random
import string
import time
import pickle

# Server communication parameters
HEADER = 64
SERVER = "0.0.0.0"
ADDR = (SERVER, PORT)
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = ['!DISCONNECT']
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(ADDR)

# Array that contain all socket connections
connections = []

# Array that contain all rooms on the server
rooms = []

# Array that contain basic stack that will be shuffle
stack = [[0,0], [1,0], [1,0], [2,0], [2,0], [3,0], [3,0], [4,0], [4,0], [5,0], [5,0], [6,0], [6,0], [7,0], [7,0], [8,0], [8,0], [9,0], [9,0], [10,0], [10,0], [11,0], [11,0], [12,0], [12,0], 
             [0,1], [1,1], [1,1], [2,1], [2,1], [3,1], [3,1], [4,1], [4,1], [5,1], [5,1], [6,1], [6,1], [7,1], [7,1], [8,1], [8,1], [9,1], [9,1], [10,1], [10,1], [11,1], [11,1], [12,1], [12,1], 
             [0,2], [1,2], [1,2], [2,2], [2,2], [3,2], [3,2], [4,2], [4,2], [5,2], [5,2], [6,2], [6,2], [7,2], [7,2], [8,2], [8,2], [9,2], [9,2], [10,2], [10,2], [11,2], [11,2], [12,2], [12,2], 
             [0,3], [1,3], [1,3], [2,3], [2,3], [3,3], [3,3], [4,3], [4,3], [5,3], [5,3], [6,3], [6,3], [7,3], [7,3], [8,3], [8,3], [9,3], [9,3], [10,3], [10,3], [11,3], [11,3], [12,3], [12,3], 
             [0,4], [0,4], [0,4], [0,4], [5,4], [5,4], [5,4], [5,4]]

# Array that contain all stacks of the server
stackByRoom = []

# Array that contain all heaps of the server
heapByRoom = []

# Array that contain all ways of the server
wayByRoom = []

#### Handle client function

This function handle the client connection, recieve and send the messages

In [3]:
def handle_client(conn, addr):
    
    # Global variable import
    global connections
    global rooms
    global stackByRoom
    global heapByRoom
    global stack
    
    # Adding the new connection to the array of connections
    connections.append(conn)
    print(f"[INFO] New connection, {addr} connected.")
    
    # Loop of the connection. While client is connected, do that :
    connected = True
    while connected:
        
        # Setting the message lenght
        msg_lenght = conn.recv(HEADER).decode(FORMAT)
        
        # If the message lenght is set, then check the messages from the client
        if msg_lenght:
            msg_lenght = int(msg_lenght)
            
            # Load the message from the client
            msg = pickle.loads(conn.recv(msg_lenght))
            
            # Receive the !DISCONNECT message
            if msg == DISCONNECT_MESSAGE:
                
                # Close the connection
                connected = False
                print(f"[INFO] A client disconnected")
                print(f"[ACTIVE CONNECTION] {threading.active_count() - 9 }")
                
                
                # Removing the player from the room where he is
                currentRoom = ""
                currentUsername = ""
                for r in rooms:
                    if r[0] == conn:
                        currentRoom = r[1]
                        currentUsername = r[2]
                for r in rooms:
                    if r[1] == currentRoom:
                        if r[0] == conn:
                            print(f"[{addr}] {currentUsername} has been diconnected from room {currentRoom}.")
                            rooms.remove(r)
                            
                # Checking if the room still exist (if there is a player in the room)
                counter = 0
                for r in rooms:
                    if r[1] == currentRoom:
                        counter = counter + 1
                        
                # If yes, send update to all the players in the room
                if counter > 0:
                    print(f"[INFO] Sending to players in room {currentRoom} updated player list.")
                    messageToSend = ["setPlayers"]
                    for r in rooms:
                        if r[1] == currentRoom:
                            messageToSend.append(r[2])
                    for r in rooms:
                        if r[1] == currentRoom and r[0] != conn:
                            r[0].send(pickle.dumps(messageToSend))
                            
                # If nobody in the room, delete all the data (stack, heap, way and room) from arrays
                elif counter == 0:
                    for s in stackByRoom:
                        if s[0] == currentRoom:
                            stackByRoom.remove(s)
                    for h in heapByRoom:
                        if h[0] == currentRoom:
                            heapByRoom.remove(h)
                    for r in rooms:
                        if r[1] == currentRoom:
                            print(f"[{addr}] Room {currentRoom} deleted.")
                            rooms.remove(r)
                    for w in wayByRoom:
                        if w[0] == currentRoom:
                            wayByRoom.remove(w)
                            print(f"[{addr}] {currentUsername} was the last player in room {currentRoom}, stack and heap for this room has been deleted.")
                
                # Checking if the room is currently playing (if yes, there are datas related to the stack of the room)
                counterStack = False
                for s in stackByRoom:
                    if s[0] == currentRoom:
                        counterStack = True
                
                # If it's the case, let's delete the room
                if counterStack == True:
                    for s in stackByRoom:
                        if s[0] == currentRoom:
                            stackByRoom.remove(s)
                    for h in heapByRoom:
                        if h[0] == currentRoom:
                            heapByRoom.remove(h)
                    for w in wayByRoom:
                        if w[0] == currentRoom:
                            wayByRoom.remove(w)
                            print(f"[{addr}] {currentUsername} disconnected room {currentRoom} that was playing. Stack and heap for this room has been deleted.")
                    roomToDelete = []
                    for r in rooms:
                        if r[1] == msg[2]:
                            roomToDelete.append(r)
                    for r in roomToDelete:
                        rooms.remove(r)
                        
            # Receive the createRoom message
            elif msg[1] == "createRoom":
                
                # Generate a random room number and send it to the player
                randomRoomNum = random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits)
                messageToSend = ['joinNow',f'{randomRoomNum}']
                rooms.append([conn, randomRoomNum, msg[0]])
                print(f"[{addr}] {msg[0]} created room number {randomRoomNum}")
                conn.send(pickle.dumps(messageToSend))
            
            # Receive the joinRoom message
            elif msg[1] == "joinRoom":
                
                # Checking if the player is already in the room
                counter = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                        if r[0] == conn:
                            print(f"[{addr}] {msg[0]} was already in room number {msg[2]}. He has been diconnected.")
                            rooms.remove(r)
                
                # Checking if the room is playing (same methode as in the disconnect message)
                counterStack = False
                for s in stackByRoom:
                    if s[0] == msg[2]:
                        counterStack = True
                        
                # If it's the case, return the room code error to the player
                if counterStack == True:
                    messageToSend = ['error','roomCodeNotValid']
                    conn.send(pickle.dumps(messageToSend))
                    print(f"[{addr}] {msg[0]} tried to join room number {msg[2]}, but room doesn't exist or is full.")
                
                # Else, check if the number of players in the room is valid
                elif counter != 0 and counter < 4:
                    
                    # Checking if the username is already used
                    counterCheckUsername = 0
                    for r in rooms:
                        if r[1] == msg[2]:
                            if r[2] == msg[0]:
                                counterCheckUsername = counterCheckUsername + 1
                    
                    # If the username isn't used and the number of players in the room is valid, join the room
                    if (counterCheckUsername == 0):
                        rooms.append([conn, msg[2], msg[0]])
                        print(f"[{addr}] {msg[0]} joined room number {msg[2]}. Sending update...")
                        messageToSend = ['joinNow',f'{msg[2]}']
                        conn.send(pickle.dumps(messageToSend))
                        messageToSend = ["setPlayers"]
                        for r in rooms:
                            if r[1] == msg[2]:
                                messageToSend.append(r[2])
                        for r in rooms:
                            if r[1] == msg[2] and r[0] != conn:
                                r[0].send(pickle.dumps(messageToSend))
                    
                    # Else, return usernameNotValid error to the player
                    else:
                        messageToSend = ['error','usernameNotValid']
                        conn.send(pickle.dumps(messageToSend))
                        print(f"[{addr}] {msg[0]} tried to join room number {msg[2]}, but username was already used.")
                
                # Else, return the room code error to the player (room already playing)
                else:
                    messageToSend = ['error','roomCodeNotValid']
                    conn.send(pickle.dumps(messageToSend))
                    print(f"[{addr}] {msg[0]} tried to join room number {msg[2]}, but room doesn't exist or is full.")
            
            # Receive the getPlayers message
            elif msg[1] == "getPlayers":
                
                # Get and send player list of the room to the client
                print(f"[INFO] {msg[0]} want players in room {msg[2]}.")
                messageToSend = ['setPlayers']
                for r in rooms:
                    if r[1] == msg[2]:
                        messageToSend.append(r[2])
                conn.send(pickle.dumps(messageToSend))
            
            # Receive the quitRoom message
            elif msg[1] == "quitRoom":
                
                # Remove the player from the room
                for r in rooms:
                    if r[1] == msg[2]:
                        if r[0] == conn:
                            print(f"[{addr}] {msg[0]} has been diconnected from room {msg[2]}.")
                            rooms.remove(r)
                
                # Check if the room still exist (user in it)
                counter = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                
                # If it's the case, send the setPlayers message (with players list) that will handle an end game on the client side
                if counter > 0:
                    print(f"[INFO] Sending to players in room {msg[2]} updated player list.")
                    messageToSend = ['setPlayers']
                    for r in rooms:
                        if r[1] == msg[2]:
                            messageToSend.append(r[2])
                    for r in rooms:
                        if r[1] == msg[2] and r[0] != conn:
                            r[0].send(pickle.dumps(messageToSend))
            
            # Receive the startGame message
            elif msg[1] == "startGame":
                
                # Checking number of players in the room and send appropriate errors
                counter = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                if counter == 0:
                    messageToSend = ['error','noRoomWithThisCode']
                    print(f"[INFO] {msg[0]} wanted to start room {msg[2]}, but there is no room with this code.")
                    conn.send(pickle.dumps(messageToSend))
                elif counter < 2:
                    messageToSend = ['error','notAValidNumberOfPerson']
                    print(f"[INFO] {msg[0]} wanted to start room {msg[2]}, but there wasn't enough players.")
                    conn.send(pickle.dumps(messageToSend))
                
                # If the number of players in the room is valid...
                else:
                    
                    # Shuffuling the stack
                    tempStack = stack[:]
                    random.shuffle(tempStack)
                    
                    # Getting current room code
                    currentRoom = ""
                    for r in rooms:
                        if r[0] == conn:
                            currentRoom = r[1]
                            
                    # Cleaning stack, heap and wayByRoom in case they exist
                    for s in stackByRoom:
                        if s[0] == currentRoom:
                            stackByRoom.remove(s)
                    for h in heapByRoom:
                        if h[0] == currentRoom:
                            heapByRoom.remove(h)
                    for w in wayByRoom:
                        if w[0] == currentRoom:
                            wayByRoom.remove(w)
                            
                    # Initialising stack and wayByRoom of the room starting
                    stackByRoom.append([msg[2],tempStack])
                    wayByRoom.append([msg[2],False])
                    for s in stackByRoom:
                        if s[0] == msg[2]:
                            roomPlaceInStack = stackByRoom.index(s)
                    
                    # Giving cards to all players (starter, 7 cards)
                    counter = 0
                    hands = []
                    for r in rooms:
                        if r[1] == msg[2]:
                            counter = counter + 1
                    for c in range(0, counter):
                        tmpList = []
                        for i in range(0, 7):
                            tmpList.append(stackByRoom[roomPlaceInStack][1].pop(0))
                        hands.append(tmpList)
                    while ((stackByRoom[roomPlaceInStack][1][0][1] == 4) | (stackByRoom[roomPlaceInStack][1][0][0] in range(10, 13))) : 
                        random.shuffle(stackByRoom[roomPlaceInStack][1])
                    
                    # Setting the heap of the room
                    heapCard = stackByRoom[roomPlaceInStack][1].pop(0)
                    heapByRoom.append([msg[2],heapCard])
                    print(f"[INFO] {msg[0]} started room {msg[2]}.")
                    counter = 0
                    
                    # Sending to the players room informations that has been setup before
                    for r in rooms:
                        if r[1] == msg[2]:
                            messageToSend = ['startNow']
                            messageToSend.append(heapCard)
                            messageToSend.append(hands[counter])
                            
                            # If the player is the owner, sending him the information of "yourTurn" (it's his turn)
                            if counter == 0:
                                messageToSend.append('yourTurn')
                                
                            r[0].send(pickle.dumps(messageToSend))
                            counter = counter + 1
            
            # Receive the placing message
            elif msg[1] == "placing":
                print(f"[INFO] {msg[0]} placed card {msg[3]} in room {msg[2]}.")
                
                # Setting up the array with all players playing in the room
                playersPlaying = []
                counter = -1
                indexCurrentPlayer = 0
                currentPlayerUsername = ""
                for r in rooms:
                    if r[1] == msg[2]:
                        playersPlaying.append(r)
                
                # Check if there is a "changement de sens". If it's the case, reverse the player array
                for w in wayByRoom:
                    if w[0] == msg[2]:
                        if w[1] == True:
                            playersPlaying.reverse()
                
                # Define current index variable of the client
                for p in playersPlaying:
                    counter = counter + 1
                    if p[2] == msg[0]:
                        indexCurrentPlayer = counter
                        
                # Initialising a variable to avoid multiple reverse of the list bug
                counterWayChanging = False
                
                # Loop to send informations to all the players in the room
                for player in playersPlaying:
                    
                    messageToSend = ['updateHeap']
                    messageToSend.append(msg[3])
                    messageToSend.append(playersPlaying[indexCurrentPlayer][2])
                    messageToSend.append(msg[4])
                    
                    # Update the heap
                    for heap in heapByRoom:
                        if heap[0] == msg[2]:
                            heap[1] = msg[3]
                            
                    # Check if card is "+2" or ""+4"
                    if msg[3][0] == 12 or (msg[3][1] == 4 and (msg[3][0] >= 6 and msg[3][0] <= 9)):
                        
                        # Skipping a player
                        if indexCurrentPlayer+2 == len(playersPlaying):
                            messageToSend.append(playersPlaying[0][2])
                            if playersPlaying.index(player) == 0:
                                messageToSend.append('yourTurn')
                        elif indexCurrentPlayer+2 > len(playersPlaying):
                            messageToSend.append(playersPlaying[1][2])
                            if playersPlaying.index(player) == 1:
                                messageToSend.append('yourTurn')
                        elif playersPlaying.index(player) == indexCurrentPlayer+2:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+2][2])
                            messageToSend.append('yourTurn')
                        else:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+2][2])
                        
                        # If the current client is the one who takes +2 or +4, giving him 2 or 4 cards
                        if indexCurrentPlayer+1 == len(playersPlaying):
                            if playersPlaying.index(player) == 0:
                                for s in stackByRoom:
                                    if s[0] == msg[2]:
                                        roomPlaceInStack = stackByRoom.index(s)
                                        
                                # Checking if there are cards remaining in the stack of the room
                                if len(stackByRoom[roomPlaceInStack][1]) <=4:
                                    tempStack = stack[:]
                                    random.shuffle(tempStack)
                                    stackByRoom[roomPlaceInStack][1] = tempStack
                                
                                # Checking if +2
                                if msg[3][0] == 12:
                                    
                                    # Giving 2 cards
                                    messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                                    messageToSend.append('takeCards')
                                else:
                                    
                                    # Giving 4 cards
                                    messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                                    messageToSend.append('takeCards')
                        
                        # If the current client is the one who takes +2 or +4, giving him 2 or 4 cards
                        elif playersPlaying.index(player) == indexCurrentPlayer+1:
                            for s in stackByRoom:
                                if s[0] == msg[2]:
                                    roomPlaceInStack = stackByRoom.index(s)
                            
                            # Checking if there are cards remaining in the stack of the room
                            if len(stackByRoom[roomPlaceInStack][1]) <= 4:
                                tempStack = stack[:]
                                random.shuffle(tempStack)
                                stackByRoom[roomPlaceInStack][1] = tempStack
                            
                            # Checking if +2
                            if msg[3][0] == 12:
                                
                                # Giving 2 cards
                                messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                                messageToSend.append('takeCards')
                            else:
                                
                                # Giving 4 cards
                                messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                                messageToSend.append('takeCards')
                    
                    # Check if card is "interdiction de jouer"
                    elif msg[3][0] == 10:
                        
                        # Skipping one player and sending "yourTurn" to player + 2
                        if indexCurrentPlayer+2 == len(playersPlaying):
                            messageToSend.append(playersPlaying[0][2])
                            if playersPlaying.index(player) == 0:
                                messageToSend.append('yourTurn')
                        elif indexCurrentPlayer+2 > len(playersPlaying):
                            messageToSend.append(playersPlaying[1][2])
                            if playersPlaying.index(player) == 1:
                                messageToSend.append('yourTurn')
                        elif playersPlaying.index(player) == indexCurrentPlayer+2:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+2][2])
                            messageToSend.append('yourTurn')
                        else:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+2][2])
                    
                    # Check if card is "changement de sens"
                    elif msg[3][0] == 11:
                        
                        # Inverting the way of the room
                        for w in wayByRoom:
                            if w[0] == msg[2] and counterWayChanging == False:
                                if w[1] == False:
                                    w[1] = True
                                    counterWayChanging = True
                                    break
                                else:
                                    w[1] = False
                                    counterWayChanging = True
                                    break
                                    
                        # Sending "yourTurn" to player - 1
                        if indexCurrentPlayer-1 < 0:
                            messageToSend.append(playersPlaying[len(playersPlaying)-1][2])
                            if playersPlaying.index(player) == len(playersPlaying)-1:
                                messageToSend.append('yourTurn')
                        elif playersPlaying.index(player) == indexCurrentPlayer-1:
                            messageToSend.append(playersPlaying[indexCurrentPlayer-1][2])
                            messageToSend.append('yourTurn')
                        else:
                            messageToSend.append(playersPlaying[indexCurrentPlayer-1][2])
                    
                    # Else, normal skip of player
                    else:
                        if indexCurrentPlayer+1 == len(playersPlaying):
                            messageToSend.append(playersPlaying[0][2])
                            if playersPlaying.index(player) == 0:
                                messageToSend.append('yourTurn')
                        elif playersPlaying.index(player) == indexCurrentPlayer+1:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+1][2])
                            messageToSend.append('yourTurn')
                        else:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+1][2])
                    
                    # Sending message
                    player[0].send(pickle.dumps(messageToSend))
            
            # Receive the demandingCard message
            elif msg[1] == "demandingCard":
                print(f"[INFO] {msg[0]} ask for a card in room {msg[2]}.")
                
                # Setting up the array with all players playing in the room
                playersPlaying = []
                counter = -1
                indexCurrentPlayer = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        playersPlaying.append(r)
                        
                # Check if there is a "changement de sens". If it's the case, reverse the player array
                for w in wayByRoom:
                    if w[0] == msg[2]:
                        if w[1] == True:
                            playersPlaying.reverse()
                
                # Define current index variable of the client
                for p in playersPlaying:
                    counter = counter + 1
                    if p[2] == msg[0]:
                        indexCurrentPlayer = counter
                        
                # Loop to send informations to all the players in the room
                for player in playersPlaying:
                    messageToSend = ['askedCard']
                    messageToSend.append(playersPlaying[indexCurrentPlayer][2])
                    messageToSend.append(msg[3])
                    
                    # Normal skip of player
                    if indexCurrentPlayer+1 == len(playersPlaying):
                        messageToSend.append(playersPlaying[0][2])
                        if playersPlaying.index(player) == 0:
                            messageToSend.append('yourTurn')
                    elif playersPlaying.index(player) == indexCurrentPlayer+1:
                        messageToSend.append(playersPlaying[indexCurrentPlayer+1][2])
                        messageToSend.append('yourTurn')
                    else:
                        messageToSend.append(playersPlaying[indexCurrentPlayer+1][2])
                    
                    # Sending to all the player...
                    if player[0] != conn:
                        player[0].send(pickle.dumps(messageToSend))
                        
                    # Excepted the one who need a new card
                    else:
                        
                        # Sending him the "newCard" message with the new card
                        messageToSend = ['newCard']
                        messageToSend.append(playersPlaying[indexCurrentPlayer][2])
                        messageToSend.append(msg[3])
                        
                        # Getting him a new card
                        for s in stackByRoom:
                            if s[0] == msg[2]:
                                roomPlaceInStack = stackByRoom.index(s)
                        if len(stackByRoom[roomPlaceInStack][1]) <= 1:
                            tempStack = stack[:]
                            random.shuffle(tempStack)
                            stackByRoom[roomPlaceInStack][1] = tempStack
                        messageToSend.append(stackByRoom[roomPlaceInStack][1].pop(0))
                        
                        # Sending next player informaiton
                        if indexCurrentPlayer+1 == len(playersPlaying):
                            messageToSend.append(playersPlaying[0][2])
                        else:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+1][2])
                        
                        # Sending through the socket
                        conn.send(pickle.dumps(messageToSend))
            
            # Receive the updateNumberOfCards message
            elif msg[1] == "updateNumberOfCards":
                print(f"[INFO] {msg[0]} sent update of his hand in room {msg[2]}.")
                
                # Sending to all the players the new number of cards of the player
                messageToSend = ['updateNumberOfCards']
                messageToSend.append(msg[0])
                messageToSend.append(msg[3])
                for r in rooms:
                    if r[1] == msg[2]:
                        r[0].send(pickle.dumps(messageToSend))
            
            # Receive the uno message
            elif msg[1] == "uno":
                print(f"[INFO] {msg[0]} sent uno message in room {msg[2]}.")
                
                # Sending to all the players the information of the player that said ICHI
                messageToSend = ['unoFromPlayer']
                messageToSend.append(msg[0])
                for r in rooms:
                    if r[1] == msg[2]:
                        r[0].send(pickle.dumps(messageToSend))
            
            # Receive the counterUno message
            elif msg[1] == "counterUno":
                print(f"[INFO] {msg[0]} sent counter uno message in room {msg[2]}.")
                
                # Getting two new cards and sending them to the player that forgot to say ICHI
                messageToSend = ['takeCardsCounterUnoHit']
                for s in stackByRoom:
                    if s[0] == msg[2]:
                        roomPlaceInStack = stackByRoom.index(s)
                if len(stackByRoom[roomPlaceInStack][1]) <=2:
                    tempStack = stack[:]
                    random.shuffle(tempStack)
                    stackByRoom[roomPlaceInStack][1] = tempStack
                messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                for r in rooms:
                    if r[1] == msg[2] and r[2] == msg[3]:
                        r[0].send(pickle.dumps(messageToSend))
            
            # Receive the winning message
            elif msg[1] == "winning":
                print(f"[INFO] {msg[0]} won the room {msg[2]}.")
                
                # Sending to all the player the winning (end game) message
                for r in rooms:
                    print("sending loop")
                    if r[1] == msg[2]:
                        print(f"sending to {r[2]}")
                        messageToSend = ['partyEnd']
                        messageToSend.append(msg[0])
                        r[0].send(pickle.dumps(messageToSend))
            
            # Receive the leaveRoom message
            elif msg[1] == "leaveRoom":
                print(f"[INFO] {msg[0]} left the room {msg[2]}.")
                
                # Sending the setPlayers message that will handle an end game on the client side
                for r in rooms:
                    if r[1] == msg[2]:
                        messageToSend = ['setPlayers']
                        messageToSend.append(msg[0])
                        r[0].send(pickle.dumps(messageToSend))
                
                # Determinating current room code
                currentRoom = ""
                for r in rooms:
                    if r[0] == conn:
                        currentRoom = r[1]
                
                # Deleting the room properly (with stack, heap and way room datas)
                for s in stackByRoom:
                    if s[0] == currentRoom:
                        stackByRoom.remove(s)
                for h in heapByRoom:
                    if h[0] == currentRoom:
                        heapByRoom.remove(h)
                for w in wayByRoom:
                    if w[0] == currentRoom:
                        wayByRoom.remove(w)
                roomToDelete = []
                for r in rooms:
                    if r[1] == msg[2]:
                        roomToDelete.append(r)
                for r in roomToDelete:
                    rooms.remove(r)
    
    # Close the connection and remove it from the table
    conn.close()
    connections.remove(conn)

#### Start function

This function is the function that start the server and handle client connections in differents "thread"

In [4]:
def start():
    server.listen()
    print(f"[LISTENING] Server is listening on {SERVER}")
    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=handle_client, args=(conn, addr))
        thread.start()
        print(f"[ACTIVE CONNECTION] {threading.active_count() - 8 }")
print("[STARTING] Sever is starting…")

[STARTING] Sever is starting…


#### Start the server

In [ ]:
start()

[LISTENING] Server is listening on 0.0.0.0
